In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("Homework2.ipynb")

# Sentiment Analysis using Naive Bayes

In [2]:
import re
import string
import random
from collections import Counter

# pip install nltk
import nltk
import pandas as pd
import numpy as np
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import otter

grader = otter.Notebook()
nltk.download('stopwords')
stop_words = stopwords.words()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lilyr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Part 1: Data Preprocessing

In [3]:
def clean_data(review):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    review = review.lower()

    
    review = emoji_pattern.sub(r'', review)   
    review=re.sub("isn't",'is not',review)
    review=re.sub("he's",'he is',review)

    #using the examples above, replace any contractions with their base words
    review = re.sub("i'm","i am",review)
    review = re.sub("you're","you are",review)
    review = re.sub("she's", "she is", review)
    review = re.sub("it's", "it is", review)
    review = re.sub("we're", "we are", review)
    review = re.sub("they're", "they are", review)
    review = re.sub("can't", "cannot", review)
    review = re.sub("don't", "do not", review)
    review = re.sub("won't", "will not", review)
    review = re.sub("shouldn't", "should not", review)
    review = re.sub("wouldn't", "would not", review)
    review = re.sub("couldn't", "could not", review)
    review = re.sub("aren't", "are not", review)
    review = re.sub("haven't", "have not", review)
    review = re.sub("hasn't", "has not", review)
    review = re.sub("hadn't", "had not", review)
    review = re.sub("wasn't", "was not", review)
    review = re.sub("weren't", "were not", review)
    review = re.sub("'ll", " will", review)
    review = re.sub("'s", " is", review)


    review = re.sub('https?://\S+|www\.\S+', '', review)

    #using the example above, remove characters such as newlines, punctuation, numbers, and other non-alphabetical characters
    review = re.sub('[^0-9a-zA-Z\s]+', '', review)
    review = re.sub('\n', '', review)
    
    return review

In [4]:
# read in and clean data
data = pd.read_csv("IMDB Dataset.csv")
cleaned_data = data['review'].apply(clean_data)

cleaned_data = pd.DataFrame(cleaned_data)
cleaned_data['sentiment'] = data['sentiment']

In [5]:
print(clean_data("That's crazy. You'll regret watching. I'm shocked. This wasn't good. There's no plot! I couldn't watch it."))
clean_data("That's crazy. You'll regret watching. I'm shocked. This wasn't good. There's no plot! I couldn't watch it.") == 'that is crazy you will regret watching i am shocked this was not good there is no plot i could not watch it'

that is crazy you will regret watching i am shocked this was not good there is no plot i could not watch it


True

In [6]:
# remove stop words
# this will take a long time to run, likely at least 10 minutes.
cleaned_data['no_sw'] = cleaned_data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [7]:
# find value counts of all words in all data points
counter = Counter()
for text in cleaned_data["no_sw"].values:
    for word in text.split():
        counter[word] += 1

# creates a set of the 10 most common words amongst all reviews
FREQWORDS = set([w for (w, wc) in counter.most_common(10)])

In [8]:
COPY_cleaned_data = cleaned_data.copy()

In [9]:
def remove_freqwords(text, freqwords):
    '''Given a review that has been cleaned, returns the review without the most frequently appearing words'''
    #complete this function which removes any of the most frequent words in a given review that do not add value
    text = ' '.join([word for word in text.split() if word not in (freqwords)])


# remove frequent words that do not add value to the model
print(FREQWORDS)
cleaned_data['no_sw'] = cleaned_data['no_sw'].apply(lambda x: ' '.join([word for word in x.split() if word not in (FREQWORDS)]))
# COPY_cleaned_data["no_sw"] = COPY_cleaned_data["no_sw"].apply(lambda text: remove_freqwords(text, FREQWORDS))
print(cleaned_data)

{'movie', 'film', 'made', 'make', 'story', 'characters', 'time', 'great', 'movies', 'br'}
                                                  review sentiment  \
0      one of the other reviewers has mentioned that ...  positive   
1      a wonderful little production br br the filmin...  positive   
2      i thought this was a wonderful way to spend ti...  positive   
3      basically there is a family where a little boy...  negative   
4      petter mattei is love in the time of money is ...  positive   
...                                                  ...       ...   
49995  i thought this movie did a down right good job...  positive   
49996  bad plot bad dialogue bad acting idiotic direc...  negative   
49997  i am a catholic taught in parochial elementary...  negative   
49998  i am going to have to disagree with the previo...  negative   
49999  no one expects the star trek movies to be high...  negative   

                                                   no_sw  
0      rev

In [10]:
# replace the sentiment with binary 0 or 1, prepare for model
#tokenized_data = cleaned_data.drop(columns=['review'])
tokenized_data = cleaned_data.drop(columns=['review'])
tokenized_data.columns = ['sentiment', 'review']
#print(tokenized_data)

#TODO - convert the sentiment values to a binary 0 or 1
tokenized_data['sentiment'] = tokenized_data['sentiment'].replace({'negative': 0, 'positive': 1})


#TODO - complete data preprocessing by changing the review format from a string of words to a list of strings containing individual words
tokenized_data['review'] = tokenized_data['review'].apply(lambda string: string.split())
print(tokenized_data)

       sentiment                                             review
0              1  [reviewers, mentioned, watching, 1, oz, episod...
1              1  [wonderful, production, filming, technique, un...
2              1  [wonderful, spend, hot, summer, weekend, sitti...
3              0  [basically, family, boy, jake, thinks, zombie,...
4              1  [petter, mattei, love, money, visually, stunni...
...          ...                                                ...
49995          1  [job, creative, original, expecting, lotta, fu...
49996          0  [plot, dialogue, acting, idiotic, directing, a...
49997          0  [catholic, taught, parochial, elementary, scho...
49998          0  [disagree, previous, comment, side, maltin, ra...
49999          0  [expects, star, trek, high, art, fans, expect,...

[50000 rows x 2 columns]


Part 2: Naive Bayes Implementation

In [11]:
class NaiveBayesModel:
    '''Class representing the implementation of the Naive Bayes model'''

    def __init__(self) -> None:
        self.occurrence_table = {}
        self.probability_table = {}
        self.labels = []

    def train_model(self, data, labels):
        '''Runs the training process for the model, building the occurrence table and probability table'''

        #complete this function which runs the entire training process of Naive Bayes
        NaiveBayesModel.build_occurrence_table(self, data, labels)
        NaiveBayesModel.build_probability_table(self)
        NaiveBayesModel.predict(self, data)


    def build_occurrence_table(self, data, labels):
        '''Private function to create the occurrence table given the training data and labels'''

        #complete this function which creates a nested dictionary table of frequencies based on the training data.
        # example format for the word 'word': {'word': {0: 3, 1: 15}}

        for index,value in data.items():
            for word in value:
                if word not in self.occurrence_table.keys():
                    self.occurrence_table[word]= {0:1, 1:1}
                    # self.occurrence_table[word][labels[index]] += 1
                else:
                    # if labels[index] not in self.occurrence_table[word].keys():
                        # self.occurrence_table[word][labels[index]] = 1
                    # else:
                    self.occurrence_table[word][labels[index]] += 1

    def build_probability_table(self):
        '''Private function to create the probability table based on the occurrence table'''

        #complete this function which generates a table of probabilities based on the frequencies recorded in the occurrence table
        # example format for the word 'word': {'word': {0: 0.1667, 1: 0.8333}}
        for word in self.occurrence_table.keys():
            self.probability_table[word] = {}
            self.probability_table[word][0] = (self.occurrence_table[word][0]/(self.occurrence_table[word][0]+self.occurrence_table[word][1]))
            # self.probability_table[word] = {}
            self.probability_table[word][1] = (self.occurrence_table[word][1]/(self.occurrence_table[word][0]+self.occurrence_table[word][1]))


    def predict(self, variables):
        '''Takes a set of variables, and predicts the class they should belong to'''

        predictions = []

        #implement prediction using the Naive Bayes method
        # remember that if the model has the same probability of either class, it should pick randomly between the two
        for index,review in variables.items():
            zero_prob = 1
            one_prob = 1
            for word in review:
                if word in self.probability_table.keys():
                    zero_prob *= self.probability_table[word][0]
                    one_prob *= self.probability_table[word][1]
            prediction = -1
            if zero_prob > one_prob:
                prediction = 0
            elif (one_prob > zero_prob):
                prediction = 1
            else:
                prediction = random.choice([zero_prob, one_prob])
            predictions.append(prediction)


        return predictions

In [12]:
# perform a train/test split on the data
X=tokenized_data['review']
y=tokenized_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=30)

# train and predict using the Naive Bayes model implementation
nb_model = NaiveBayesModel()
nb_model.train_model(X_train, y_train)
pred = nb_model.predict(X_test)

In [13]:
#Create and display confusion matrices and evaluation metrics to prove the performance of your model.
# required: accuracy, recall, f-score

# confusion matrix

confusion_matrix = sklearn.metrics.confusion_matrix(y_test,pred)
print(confusion_matrix)

# accuracy
print((pred==y_test).sum()/len(pred))
# recall
print((confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[1][0])))
# f-score
print((confusion_matrix[0][0]/(confusion_matrix[0][0] + 0.5 * (confusion_matrix[0][1] + confusion_matrix[1][0]))))


[[3944 1033]
 [ 412 4611]]
0.8555
0.9054178145087236
0.8451730418943534


## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [15]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)

ValueError: Could not resolve notebook path